Define data. The index_col = 0 indicates that the row names are in the first column.  The function read_excel() assumes that the first row is column names.  price_data.index gives the row names of the data frame price_data.

In [1]:
import pandas as pd

inventory_data = pd.read_excel("RedTomatoes.xlsx",
                          sheet_name="Sheet1",
                          index_col=0)
M = list(inventory_data.index)
print(M)

Mplus0 = M.copy() # make a copy of M
Mplus0.insert(0,0) # insert month 0 in place 0
print(Mplus0)

demand = dict(zip(M, inventory_data.demand))

print(demand)


['January', 'February', 'March', 'April', 'May', 'June']
[0, 'January', 'February', 'March', 'April', 'May', 'June']
{'January': 1600, 'February': 3000, 'March': 3200, 'April': 3800, 'May': 2200, 'June': 2200}


Import Pyomo and declare a model object.

In [2]:
from pyomo.environ import *

model = ConcreteModel()

Declare nonnegative variables for amount produced each month.

In [3]:
try: 
    model.del_component(model.x)
    model.del_component(model.x_index)
except:
    print("Defining variables for amount produced each month first time")
model.x = Var(M,domain=NonNegativeReals)


Defining variables for amount produced each month first time


Define nonnegative variables for inventory.

In [4]:
try:
    model.del_component(model.inventory)
    model.del_component(model.inventory_index)
except:
    print("Defining variables for first time")
model.inventory = Var(M,domain=NonNegativeReals)


Defining variables for first time


Define nonnegative variables for inventory.

Define the objective function.

In [5]:
try:
    model.del_component(model.TotalCost)
except:
    print("Defining Total Cost objective for the first time")
model.TotalCost = Objective(
    expr = sum(640workforce[i] for i in M) + sum(60overtime[i] for i in M)+ sum(300hired[i] for i in M)+ sum(500layoff[i] for i in M)+ sum(2inventory[i] for i in M)+ sum(5stockout[i] for i in M)+ sum(10X[i] for i in M)+ sum(30subcontract[i] for i in M),
    sense=minimize)
model.pprint()

SyntaxError: invalid syntax (<ipython-input-5-36338733af17>, line 6)

Inventory balance constraints.

In [0]:
try:
    model.del_component(model.InventoryBalance)
    model.del_component(model.InventoryBalance_index)
except:
    print("Defining InventoryBalance for the first time")
model.InventoryBalance = ConstraintList()

for i in M:
    model.InventoryBalance.add(inventory[i-1] + reg[i] + overtime[i] + subcontract[i] - demand[i] - stockout[i-1] == inventory[i] - stockout[i])
model.pprint()

Initial inventory constraint.

In [7]:
try:
    model.del_component(model.InitialInventory)
except:
    print("Defining InitialInventory for the first time")
model.InitialInventory = Constraint(
    expr = model.InitialInventory[0] == 1000)
model.pprint()

Defining InitialInventory for the first time


KeyError: "Index '0' is not valid for indexed component 'y'"

Defining Initial workforce for the first time

In [0]:
try:
    model.del_component(model.InitialWorkforce)
except:
    print("Defining InitialWorkforce for the first time")
model.InitialWorkforce = Constraint(
    expr = model.InitialWorkforce[0] == 80)
model.pprint()

Capacity constraint

In [8]:
try:
    model.del_component(model.CapacityBalance)
    model.del_component(model.CapacityBalance_index)
except:
    print("Defining CapacityBalance for the first time")
model.CapacityBalance = ConstraintList()

for i in M:
    model.CapacityBalance.add(model.x[i] <= 40workforce[i] + (overtime[i]/4))
model.pprint()

SyntaxError: invalid syntax (<ipython-input-8-6b96942df86b>, line 9)

Overtime Limit constraint

In [9]:
try:
    model.del_component(model.OvertimeLimit)
    model.del_component(model.OvertimeLimit_index)
except:
    print("Defining Overtime Limit for the first time")
model.OvertimeLimit = ConstraintList()

for i in M:
    model.OvertimeLimit.add(overtime[i] <= 10workforce[i])
model.pprint()

SyntaxError: invalid syntax (<ipython-input-9-21804113d849>, line 9)

Specify solver and solve.


In [10]:
solver = SolverFactory('glpk')
status = solver.solve(model)

ValueError: ERROR: No objectives defined for input model. Cannot write legal LP file.

Get status.

In [0]:
print('Status = %s' % status)

Get solution and objective function value.

In [5]:
for i in M:
    print('%s = %f, %s = %f' % (model.x[i], value(model.x[i]), model.inventory[i], value(model.inventory[i])))
print("Objective = %f" % value(model.TotalCost))

ERROR: evaluating object as numeric value: x[January]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[January]


ValueError: No value for uninitialized NumericValue object x[January]

The optimal solution is to produce 1150 units in month 1 and 1700 in month 3.  Hold 650 in inventory after month 1, and hold 700 in inventory after month 3.  The total cost is $152500.